## Exercise 05.1 (random numbers)

- Using the `randint` function from the `random` module (https://docs.python.org/3/library/random.html#random.randint) to
  develop a function `dice_roll` that emulates the roll of a dice with $n$ sides. The number of sides `n` should an argument to the function.

- For $n=6$, devise and implement a test to check that it is a fair dice.

#### (a) Dice roll code:

In [2]:
import random

def dice_roll(n):
    """Return a random roll of a dice with n sides."""
    return random.randint(1, n)

dice_roll(6)

2

In [3]:
## tests ##
for n in range(1, 20):
    for j in range(100):
        value = dice_roll(n) 
        assert value >= 1 and value <= n

#### (b) Test for fairness

In [13]:
# To check for fairness, we roll the dice a large number of times and check how many times we get each value. 
# We expect to get each value close to $1/6$ of the times. 
import random
from collections import Counter  

def test_fairness(n, rolls):
    results = [dice_roll(n) for _ in range(rolls)]
    counts = Counter(results)
    
    print("Results of dice rolls:")
    for side in range(1, n+1):
        print(f"Side {side}: {counts[side]} ({counts[side]/rolls:.2%})")

    # Check if distribution is uniform
    expected = rolls / n
    tolerance = 0.05 * expected 
    fair = all(abs(counts[side] - expected) <= tolerance for side in range(1, n+1))
    
    print("\nDice fairness:", "Fair" if fair else "Not fair")

# Example test run
test_fairness(6, 60000)

Results of dice rolls:
Side 1: 9911 (16.52%)
Side 2: 10001 (16.67%)
Side 3: 10116 (16.86%)
Side 4: 10022 (16.70%)
Side 5: 10060 (16.77%)
Side 6: 9890 (16.48%)

Dice fairness check: Fair


## Exercise 05.2 (variance estimation)

For a random variable $X$, the variance of $X$ is defined as  

$$
\begin{align}
\mathrm{Var}\left[ X \right] &= \mathrm{E}\left[\left( X - \mu \right)^2\right]  \\ 
&= \mathrm{E}\left[ X^{2} \right] - \mathrm{E}\left[ X \right]^2
\end{align}
$$

where $\mathrm{E}$ is the 'expectation' (mean of something) and $\mu = \mathrm{E}(X)$ is the mean of $X$. If we have all data (the entire 'population'),
the variance can be computed from:

$$
\mathrm{Var}\left[ X \right] = \frac{\sum_{i=0}^{N-1} x^{2}_{i}}{N} - \left( \frac{ \left( \sum_{i=0}^{N-1} x_{i} \right)}{N} \right)^{2}
$$

Often, we only have a sample of data. For example, we might want to estimate the variance in height for students at a university using just a random sample of students. 
When using a sample from a larger data set to estimate the variance, the above formula has a *bias*. Therefore, it is common to use the *unbiased* estimator

$$
s^{2} = \left( \frac{\sum_{i=0}^{n-1} x^{2}_{i}}{n} - \left( \frac{ \left( \sum_{i=0}^{n-1} x_{i} \right)}{n} \right)^{2} \right) \frac{n}{n-1}
$$

to estimate the variance. In this exercise we will use the unbiased estimator.

1. Create a function that returns the estimated variance for a list of numbers based on the above equation. Test your function using 1 million samples drawn from a Gaussian distribution with a mean of 10 and a standard deviation of 3. Use your crsid to seed the random number generator
   
   *See hint below on how to create the sample.*

2. For a sample drawn from a distribution with mean $5 \times 10^6$ and standard deviation $2.0$, estimate the variance using (i) your function for estimating the variance, and (ii) using the `variance` function from the Python `statistics` module. Comment on and explain any significant differences in the results from (i) and (ii).

### Hint: sampling from a distribution

The function `random.gauss` can be used to sample a Gaussian distribution with a specified mean and standard deviation (square root of the variance) *N* times, e.g.:

In [ ]:
import random

random.seed("gnw20")  # See the random number generator

mu = 10.0    # mean of the distribution
sigma = 3.0  # Standard deviation
x = [random.gauss(mu, sigma) for i in range(8)]
print(x)

### Solution

1. Estimate variance

In [15]:
import random

random.seed("cys38")  

mu = 10.0              # mean of the distribution
sigma = 3.0            # standard deviation
N = 1_000_000          # number of samples


x = [random.gauss(mu, sigma) for i in range(N)]


def estimate_variance(data):
    n = len(data)
    sum_x = sum(data)
    sum_x2 = sum(i**2 for i in data)
    # unbiased variance formula
    return (sum_x2 / n - (sum_x / n)**2) * n / (n - 1)


estimated_var = estimate_variance(x)
print("Estimated variance:", estimated_var)

print("Theoretical variance (σ²):", sigma**2)




Estimated variance: 8.990185702179557
Theoretical variance (σ²): 9.0


2. Use the `statistics` module to estimate the variance, and compare the estimated variance using `statistics` to the estimated variance using your implementation for the variance estimation.

In [19]:
import random
import statistics
import math

random.seed("gnw20")
mu = 10.0
sigma = 3.0
N = 1_000_000

x = [random.gauss(mu, sigma) for _ in range(N)]

def estimate_variance(data):
    n = len(data)
    sum_x = sum(data)
    sum_x2 = sum(v**2 for v in data)
    return (sum_x2 / n - (sum_x / n)**2) * n / (n - 1)   # unbiased (n-1)


my_var = estimate_variance(x)
stats_var = statistics.variance(x)       # unbiased (n-1)  

print(f"Unbiased variance:          {my_var:.10f}")
print(f"statistics.variance (unbiased): {stats_var:.10f}")
print(f"Abs diff (mine vs stats):       {abs(my_var - stats_var):.10e}")

print(f"\nTheoretical σ²:                  {sigma**2:.10f}")
print(f"Error (unbiased vs σ²):         {abs(stats_var - sigma**2):.10f}")


Unbiased variance:          9.0021132388
statistics.variance (unbiased): 9.0021132388
Abs diff (mine vs stats):       7.1054273576e-15

Theoretical σ²:                  9.0000000000
Error (unbiased vs σ²):         0.0021132388


## Exercise 05.03 (optional, parallel processing)

Almost all modern computer processing units have multiple processing *cores*. To utilise the full performance of a processor, operations need to be performed in *parallel*, i.e. each processing core is given a task to perform.

Parallel computing is a very rich and technical area. To help exploit multi-core systems there are libraries that support parallel processing. Below is a simple example using the Python `multiprocessing` library.

In [ ]:
import multiprocessing

# On some operaring systems, you may need to uncomment the below line
# multiprocessing.set_start_method('fork')

def f(task):
    """A function that print the input argument and the id for the process that executes the function"""
    print(f"Task index {task}, process id: {multiprocessing.current_process()}\n")
     
    return str(multiprocessing.current_process())
        
        
# Using 3 'processes', execute the function 'f' four times (each time with a different argument)  
with multiprocessing.Pool(processes=3) as p:
    procs = p.map(f, [0, 1, 2, 3])  # Call function mysort three times
    
    print("Returned data (a list)") 
    print(procs)

Consider the below code that creates a list of lists of sorted numbers.

In [ ]:
def mysort(N):
    """Create a randomly ordered list of integers of length N, and return the sorted list"""
    # Create randomly ordered list
    x = random.sample(range(0, N), N)
 
    # Return sorted list of numbers
    return sorted(x)

def sorted_lists(N, p):
    """Create a list of sorted lists"""
    data = []
    for i in range(p):
        data.append(mysort(N))

%time x = sorted_lists(1000000, 5)

Use the `multiprocessing` module to perform the above operation in parallel. Investigate how the processing time changes with the number of processes, and in particular the average time per 'create and sort operation' when changing the number of processes.

In [ ]:
...